In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [92]:
# Step 1: Load the data (assuming it's in CSV format)
df=pd.read_csv('tappy.csv')

C:\Users\Bobi\AppData\Local\Temp\ipykernel_7084\2660723381.py:1: DtypeWarning: Columns (1,4,6,7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('tappy.csv')


In [93]:
df.head(2)

,userkey,date,timestamp,hand,holdtime,direction,latencytime,flighttime,BirthYear,Gender,Parkinsons
0,pacient1-0QAZFRHQHW,160916,20:43.9,L,78.1,LL,312.5,257.8,1959.0,Female,False
1,pacient1-0QAZFRHQHW,160916,20:44.3,L,78.1,LL,453.1,375,1959.0,Female,False


In [94]:
df.tail(2)

,userkey,date,timestamp,hand,holdtime,direction,latencytime,flighttime,BirthYear,Gender,Parkinsons
1048573,pacient26-COK8G1W7JY,161121,27:33.0,R,66.4,RR,226.6,148.4,NaN,Male,False
1048574,pacient26-COK8G1W7JY,161121,27:33.3,L,78.1,RL,320.3,253.9,NaN,Male,False


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   userkey      1048575 non-null  object 
 1   date         1048575 non-null  object 
 2   timestamp    1048575 non-null  object 
 3   hand         1048575 non-null  object 
 4   holdtime     1048575 non-null  object 
 5   direction    1048575 non-null  object 
 6   latencytime  1048575 non-null  object 
 7   flighttime   1048575 non-null  object 
 8   BirthYear    727593 non-null   float64
 9   Gender       1048569 non-null  object 
 10  Parkinsons   1048569 non-null  object 
dtypes: float64(1), object(10)
memory usage: 88.0+ MB


In [96]:
df.describe()

,BirthYear
count,727593.000000
mean,1948.673170
std,4.069903
min,1936.000000
25%,1946.000000
50%,1950.000000
75%,1950.000000
max,1967.000000


In [97]:
df.isnull().sum()

userkey             0
date                0
timestamp           0
hand                0
holdtime            0
direction           0
latencytime         0
flighttime          0
BirthYear      320982
Gender              6
Parkinsons          6
dtype: int64

In [204]:
#df.fillna(df.mean(), inplace=True)


In [98]:
# Step 2: Preprocessing the data
df.dropna(inplace = True)

In [99]:
df.isnull().sum()

userkey        0
date           0
timestamp      0
hand           0
holdtime       0
direction      0
latencytime    0
flighttime     0
BirthYear      0
Gender         0
Parkinsons     0
dtype: int64

In [100]:
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})

In [101]:
df['hand'] = df['hand'].map({'L': 0, 'R': 1})

In [102]:
df['Parkinsons'].unique()

array([False, True, 'FALSE', 'TRUE', 'True '], dtype=object)

In [103]:
def mapping_parkinsons(x):
    # If the value is a boolean, return 1 for True and 0 for False
    if isinstance(x, bool):
        return int(x)  # True becomes 1, False becomes 0
    # For string values, normalize them to lowercase and strip spaces
    x = str(x).strip().lower()
    
    if x == 'false':
        return 0
    elif x == 'true':
        return 1
    else:
        return None  # Optional: handle invalid cases
    
    # Apply the function to the 'Parkinsons' column
df['Parkinsons'] = df['Parkinsons'].apply(mapping_parkinsons)

In [104]:
# Convert 'holdtime' to numeric, coercing errors to NaN
df['holdtime'] = pd.to_numeric(df['holdtime'], errors='coerce')

# Fill NaN values with a default value, e.g., 0
df['holdtime'].fillna(0, inplace=True)

# Round the values and convert to integers
df['holdtime'] = df['holdtime'].round().astype(int)

In [105]:
# Convert 'holdtime' to numeric, coercing errors to NaN
df['latencytime'] = pd.to_numeric(df['latencytime'], errors='coerce')

# Fill NaN values with a default value, e.g., 0
df['latencytime'].fillna(0, inplace=True)

# Round the values and convert to integers
df['latencytime'] = df['latencytime'].round().astype(int)

In [106]:
# Convert 'holdtime' to numeric, coercing errors to NaN
df['flighttime'] = pd.to_numeric(df['flighttime'], errors='coerce')

# Fill NaN values with a default value, e.g., 0
df['flighttime'].fillna(0, inplace=True)

# Round the values and convert to integers
df['flighttime'] = df['flighttime'].round().astype(int)

In [107]:
# Convert the 'timestamp' column to a datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%H:%M.%S', errors='coerce')

# Extract the hour, minute, and second as separate features
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['second'] = df['timestamp'].dt.second + df['timestamp'].dt.microsecond / 1e6

In [109]:
label_encoder = LabelEncoder()
df['direction'] = label_encoder.fit_transform(df['direction'])

In [128]:
df.head(5)

,userkey,date,timestamp,hand,holdtime,direction,latencytime,flighttime,BirthYear,Gender,Parkinsons,hour,minute,second
0,pacient1-0QAZFRHQHW,160916,1900-01-01 20:43:09,0,78,8,312,258,1959,0,0,20,43,9
1,pacient1-0QAZFRHQHW,160916,1900-01-01 20:44:03,0,78,8,453,375,1959,0,0,20,44,3
2,pacient1-0QAZFRHQHW,160916,1900-01-01 20:44:07,0,62,8,414,336,1959,0,0,20,44,7
3,pacient1-0QAZFRHQHW,160916,1900-01-01 20:45:05,0,94,8,742,680,1959,0,0,20,45,5
4,pacient1-0QAZFRHQHW,160916,1900-01-01 20:46:00,1,102,9,523,430,1959,0,0,20,46,0


In [111]:
# Step 3: Split the data into training and testing sets
fetures = ['hand','holdtime','direction','latencytime','flighttime','BirthYear','Gender','Parkinsons','hour','minute','second']

In [134]:
df.drop(columns='timestamp', inplace=True)

In [135]:
X = df[fetures]

In [136]:
y = df['Parkinsons']

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [138]:
# Step 4: Standardize the data (optional but recommended for SVM and Logistic Regression)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [139]:
df.dtypes

userkey        object
date           object
hand            int32
holdtime        int32
direction       int32
latencytime     int32
flighttime      int32
BirthYear       int32
Gender          int64
Parkinsons      int64
hour            int32
minute          int32
second          int32
dtype: object

In [140]:
for column in df.columns:
    try:
        # Try converting the column to numeric
        pd.to_numeric(df[column])
        print(f"{column} is numeric.")
    except ValueError as e:
        print(f"Error in column '{column}': {e}")

Error in column 'userkey': Unable to parse string "pacient1-0QAZFRHQHW" at position 0
Error in column 'date': Unable to parse string "6LB9FQABZQ" at position 130031
hand is numeric.
holdtime is numeric.
direction is numeric.
latencytime is numeric.
flighttime is numeric.
BirthYear is numeric.
Gender is numeric.
Parkinsons is numeric.
hour is numeric.
minute is numeric.
second is numeric.


In [118]:
# Check if there are NaN values in the 'hand' column
print(df['hand'].isna().sum())

# Fill NaN values in the 'hand' column with a default value (e.g., 0 or another appropriate value)
df['hand'].fillna(0, inplace=True)

# Convert the 'hand' column to integers
df['hand'] = df['hand'].astype(int)

125381


In [119]:
# Check if there are NaN values in the 'hand' column
print(df['BirthYear'].isna().sum())

# Fill NaN values in the 'hand' column with a default value (e.g., 0 or another appropriate value)
df['BirthYear'].fillna(0, inplace=True)

# Convert the 'hand' column to integers
df['BirthYear'] = df['BirthYear'].astype(int)

0


In [120]:
# Check if there are NaN values in the 'hand' column
print(df['hour'].isna().sum())

# Fill NaN values in the 'hand' column with a default value (e.g., 0 or another appropriate value)
df['hour'].fillna(0, inplace=True)

# Convert the 'hand' column to integers
df['hour'] = df['hour'].astype(int)

439468


In [121]:
# Check if there are NaN values in the 'hand' column
print(df['minute'].isna().sum())

# Fill NaN values in the 'hand' column with a default value (e.g., 0 or another appropriate value)
df['minute'].fillna(0, inplace=True)

# Convert the 'hand' column to integers
df['minute'] = df['minute'].astype(int)

439468


In [122]:
# Check if there are NaN values in the 'hand' column
print(df['second'].isna().sum())

# Fill NaN values in the 'hand' column with a default value (e.g., 0 or another appropriate value)
df['second'].fillna(0, inplace=True)

# Convert the 'hand' column to integers
df['second'] = df['second'].astype(int)

439468


In [123]:
# Check for infinity or -infinity in the DataFrame
print(df[df.isin([np.inf, -np.inf]).any(axis=1)])

Empty DataFrame
Columns: [userkey, date, timestamp, hand, holdtime, direction, latencytime, flighttime, BirthYear, Gender, Parkinsons, hour, minute, second]
Index: []


In [124]:
# Replace infinity values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [125]:
# Select only numeric columns
numeric_cols = df.select_dtypes(include=[np.number])

# Check for extremely large values in numeric columns
large_values = numeric_cols[(numeric_cols > np.finfo(np.float32).max).any(axis=1)]

In [141]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)

In [142]:
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Classification Report:\n", classification_report(y_test, y_pred_dt))

Decision Tree Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     24891
           1       1.00      1.00      1.00    120628

    accuracy                           1.00    145519
   macro avg       1.00      1.00      1.00    145519
weighted avg       1.00      1.00      1.00    145519



In [143]:
# Step 5: Train and evaluate a machine learning model
# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)

In [144]:
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print("Classification Report:\n", classification_report(y_test, y_pred_log_reg))

Logistic Regression Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     24891
           1       1.00      1.00      1.00    120628

    accuracy                           1.00    145519
   macro avg       1.00      1.00      1.00    145519
weighted avg       1.00      1.00      1.00    145519



In [148]:
# Support Vector Classifier (SVM)
svm_clf = SVC()
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)

In [149]:
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))

SVM Accuracy: 1.0
